In [1]:
import MeCab as mc
import re
import urllib.request
import unicodedata
import pandas as pd
import numpy as np
import math

In [13]:
#MeCabによる形態素解析
def extractter(text):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if nes[0] == u"名詞":
                if nes[6] == u"*":
                     keyword.append(wor)
                else:
                    keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

#ストップワードの除去
def except_stopwords(text):
    stopwords = get_stopword()
    for i in text:
        for j in stopwords:
            if i == j:
                text.remove(j)
    return text

In [3]:
#データの確認
base_data = pd.read_csv(filepath_or_buffer="Datas/pickup_data.csv", encoding="utf_8", sep=",")
base_data.info()
base_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
screen_name    20000 non-null object
user_id        20000 non-null float64
tweet_id       20000 non-null float64
tweet          20000 non-null object
tweet2         20000 non-null object
postdate       20000 non-null object
cos_day        20000 non-null float64
sin_day        20000 non-null float64
tag            20000 non-null object
image_url      20000 non-null object
image          20000 non-null int64
retweet        20000 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 1.8+ MB


,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet
0,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,固定 資産 税 各種 申告 固定 資産 税 毎年 日 現在 資産 所有 状況 課税 該当 申...,2015-12-01,-0.493776,0.869589,0,0,0,0.0
1,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,広報 平成 号 広報 まき 月 日 号 発行 内容 花巻 ホームページ ご覧,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,0.0
2,遠野市観光協会/Tono Tourist,2.324801e+09,6.715386e+17,昨日のこと よし屋 さんで遅っそいお昼ご飯 落ち着いた雰囲気ののんびりできる喫茶店 です お...,昨日 屋 お昼 ご飯 雰囲気 喫茶店 お茶 近所 おば たち 会話 盗み聞き 基本 月 営業,2015-12-01,-0.493776,0.869589,"遠野,珈琲,よし屋,壺が出てたらお店あいてます",http://pbs.twimg.com/media/CVHI0LqUkAEfSlx.jpg...,1,0.0
3,さんてつくん @ 三陸鉄道【鉄ログ】,1.335826e+08,6.715527e+17,ランチｏｒスイーツ列車 ミ,ランチ or スイーツ 列車 ミ,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVHVqayVEAEna15.jpg,1,0.0
4,花巻市,2.843129e+09,6.716279e+17,ルンビニー苑利用者の方々からシクラメンをいただきました 花巻市石鳥谷町の障がい者支援施...,ルンビニー 苑 利用 方々 シクラメン 花巻 石鳥谷 支援 施設 ルンビニー 苑 施設 利用...,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVIaB-xU8AAcsiE.jpg...,1,0.0


In [4]:
#user_idの固有データの取り出し
nu = base_data['user_id'].unique()
print(nu)
#データフレームからuser_idを抽出
ids = base_data['user_id']
#0~31の数値へ変換
post_user = []
for i in ids:
    for j in range(len(nu)):
        if i == nu[j]:
            post_user.append(j)

#データフレームへ変換
s = pd.DataFrame(post_user, columns=['user_id2'])
#元のデータと結合
#n_base_data = base_data.join(s)

[2.84312922e+09 2.32480106e+09 1.33582577e+08 1.37168394e+09
 1.02971993e+09 8.52638190e+07 2.22008119e+08 1.96899097e+08
 2.75767513e+09 2.45955083e+09 2.70280216e+08 2.64482835e+08
 4.56310285e+09 7.60852304e+08 9.56572308e+08 1.61550679e+09
 1.73378242e+09 7.40050123e+17 1.02589448e+08 7.48485695e+17
 9.71187020e+07 7.58099901e+17 1.66203998e+09 1.29824840e+08
 5.82425732e+08 3.13971513e+09 5.43138610e+08 3.01089352e+09
 8.93037779e+17 9.45626465e+17 9.73451829e+17 9.95901925e+17]


In [14]:
#不要な文字の削除
tag_list = base_data["tag"].str.replace(r'[0-9]', ' ').str.replace(r'[０-９]', ' ').str.replace(r'([^\s\w])+', ' ')
#形態素解析
for i in range(len(tag_list)):
    tag_list[i] = extractter(tag_list[i])
    tag_list[i] = except_stopwords(tag_list[i])

In [52]:
tag_list[2]

['遠野', '珈琲', 'よし', '壺']

In [54]:
#元のデータと結合
n_base_data = base_data.join(s)
n_base_data['tag2'] = tag_list

In [59]:
n_base_data.head()

,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet,tag2,user_id2
0,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,固定 資産 税 各種 申告 固定 資産 税 毎年 日 現在 資産 所有 状況 課税 該当 申...,2015-12-01,-0.493776,0.869589,0,0,0,0.0,[],0
1,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,広報 平成 号 広報 まき 月 日 号 発行 内容 花巻 ホームページ ご覧,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,0.0,[],0
2,遠野市観光協会/Tono Tourist,2.324801e+09,6.715386e+17,昨日のこと よし屋 さんで遅っそいお昼ご飯 落ち着いた雰囲気ののんびりできる喫茶店 です お...,昨日 屋 お昼 ご飯 雰囲気 喫茶店 お茶 近所 おば たち 会話 盗み聞き 基本 月 営業,2015-12-01,-0.493776,0.869589,"遠野,珈琲,よし屋,壺が出てたらお店あいてます",http://pbs.twimg.com/media/CVHI0LqUkAEfSlx.jpg...,1,0.0,"[遠野, 珈琲, よし, 壺]",1
3,さんてつくん @ 三陸鉄道【鉄ログ】,1.335826e+08,6.715527e+17,ランチｏｒスイーツ列車 ミ,ランチ or スイーツ 列車 ミ,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVHVqayVEAEna15.jpg,1,0.0,[],2
4,花巻市,2.843129e+09,6.716279e+17,ルンビニー苑利用者の方々からシクラメンをいただきました 花巻市石鳥谷町の障がい者支援施...,ルンビニー 苑 利用 方々 シクラメン 花巻 石鳥谷 支援 施設 ルンビニー 苑 施設 利用...,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVIaB-xU8AAcsiE.jpg...,1,0.0,[],0


In [69]:
#データ一時保存
n_base_data.to_csv("Datas/multi_data.csv",index=False, sep=",")
#データの確認
b_data = pd.read_csv(filepath_or_buffer="Datas/multi_data.csv", encoding="utf_8", sep=",")
b_data.info()
b_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
screen_name    20000 non-null object
user_id        20000 non-null float64
tweet_id       20000 non-null float64
tweet          20000 non-null object
tweet2         20000 non-null object
postdate       20000 non-null object
cos_day        20000 non-null float64
sin_day        20000 non-null float64
tag            20000 non-null object
image_url      20000 non-null object
image          20000 non-null int64
retweet        20000 non-null float64
tag2           20000 non-null object
user_id2       20000 non-null int64
dtypes: float64(5), int64(2), object(7)
memory usage: 2.1+ MB


,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet,tag2,user_id2
0,花巻市,2.843129e+09,6.714906e+17,固定資産税に関する各種申告等について 固定資産税は 毎年 月 日現在の資産の所有状況によ...,固定 資産 税 各種 申告 固定 資産 税 毎年 日 現在 資産 所有 状況 課税 該当 申...,2015-12-01,-0.493776,0.869589,0,0,0,0.0,[],0
1,花巻市,2.843129e+09,6.714914e+17,広報はなまき 平成 年 月 日号 広報はなまき 月 日号を発行しました ...,広報 平成 号 広報 まき 月 日 号 発行 内容 花巻 ホームページ ご覧,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVGd144U8AIKEes.jpg,1,0.0,[],0
2,遠野市観光協会/Tono Tourist,2.324801e+09,6.715386e+17,昨日のこと よし屋 さんで遅っそいお昼ご飯 落ち着いた雰囲気ののんびりできる喫茶店 です お...,昨日 屋 お昼 ご飯 雰囲気 喫茶店 お茶 近所 おば たち 会話 盗み聞き 基本 月 営業,2015-12-01,-0.493776,0.869589,"遠野,珈琲,よし屋,壺が出てたらお店あいてます",http://pbs.twimg.com/media/CVHI0LqUkAEfSlx.jpg...,1,0.0,"['遠野', '珈琲', 'よし', '壺']",1
3,さんてつくん @ 三陸鉄道【鉄ログ】,1.335826e+08,6.715527e+17,ランチｏｒスイーツ列車 ミ,ランチ or スイーツ 列車 ミ,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVHVqayVEAEna15.jpg,1,0.0,[],2
4,花巻市,2.843129e+09,6.716279e+17,ルンビニー苑利用者の方々からシクラメンをいただきました 花巻市石鳥谷町の障がい者支援施...,ルンビニー 苑 利用 方々 シクラメン 花巻 石鳥谷 支援 施設 ルンビニー 苑 施設 利用...,2015-12-01,-0.493776,0.869589,0,http://pbs.twimg.com/media/CVIaB-xU8AAcsiE.jpg...,1,0.0,[],0


In [70]:
#不要文字の置き換え
b_data['tag2'] = b_data['tag2'].str.replace(r'([^\s\w])+', '')

In [73]:
#データ再保存
b_data.to_csv("Datas/multi_data.csv",index=False, sep=",")